<a href="https://colab.research.google.com/github/Sam-Mucyo/emotion-detector/blob/main/Facial_Emotion_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Facial Emotion Detector Code Repo**


We start by making sure that deepface library is correctly installed

In [ ]:
! pip install deepface

Importing the library

In [ ]:
from deepface import DeepFace

We define a function to open a webcam and take a picture; the picture get saved in the directory so that we can later use it for emotion detection.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

Now that we have that function defined. We can use it to take a photo. The program will ask access to camera. After allowing access, you can take a photo demonstrating kind of facial expression - happy (smiling), excited, angry, disgust, et.- and the program will predict what your facial expression conveys.

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

Let's now see the results: Click run for below codes to see the predicted dominant facial emotion

In [ ]:
results = DeepFace.analyze(img_path = '/content/photo.jpg', actions=('emotion', 'age', 'gender', 'race'))
print(f"\n\nThe dominant facial emotion is: {results['dominant_emotion']}")

Run the following code to see a full report with percentages

In [ ]:
emotions = results['emotion']
print ("Full stats: \n")
for emotion in emotions:
  print (f"{emotion} = {round(emotions[emotion], 2)}%")

**TESTING WITH MORE THAN 1 IMAGES FROM A FOLDER**

When I was testing this, Google Colab couldn't allow me to upload a directory/folder. You might need to select all images, drag, and drop them in Colab (instead of uploading the entire folder). Or just create a new folder in Colab and then upload the images in that new folder.

Usage: You'll be prompted to enter the name of a folder with images you want to test. If you didn't create a folder and just uploaded images, ignore the prompt and just hit enter.

In [ ]:
from pathlib import Path
import glob
import os

current_path = os.path.expanduser('/content/')
print("If you have just uploaded images, without creating a folder, ignore the prompt and just hit enter.\n")
new_folder = input("Name of the folder you with images: ")
images = glob.glob(current_path + f'/{new_folder}' '/*.png')

for image_path in images:
  results = DeepFace.analyze(img_path = image_path, actions=('emotion', '', '', ''))
  print (f"\n\nThe dominant facial emotion is: {results['dominant_emotion']}")
  img_name = image_path.replace("/content/", "")
  display(Image(img_name))
